In [18]:
from datetime import timedelta, date
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import *
import pyspark.sql.functions as func
import time


In [19]:
##Fucntion for chaging the datatype
def data_type_change_int(df,cols):
    temp = df
    for col in cols:
        temp = temp.withColumn(col+'_1',df[col].cast(IntegerType())).drop(col).withColumnRenamed(col+'_1',col)
    return temp

def data_type_change_float(df,cols):
    temp = df
    for col in cols:
        temp = temp.withColumn(col+'_1',df[col].cast(DoubleType())).drop(col).withColumnRenamed(col+'_1',col)
    return temp

In [20]:
##los bucket for search los
def searchToCategory(srch_los,):
    if srch_los in (0,1,2,3,4,5,6,7):
        return str(srch_los)
    elif srch_los >= 8 and srch_los <= 14:
        return '8-14'
    else:
        return '>14'

##bw bucket for srch bw 
def searchbucketToCategory(SRCH_WINDOW):
    if SRCH_WINDOW <= 0:
        return '<=0'
    elif SRCH_WINDOW == 1:
        return '1'
    elif SRCH_WINDOW == 2:
        return '2'
    elif SRCH_WINDOW >= 3 and SRCH_WINDOW <= 4:
        return '3-4'
    elif SRCH_WINDOW >= 5 and SRCH_WINDOW <= 7:
        return '5-7'
    elif SRCH_WINDOW >= 8 and SRCH_WINDOW <= 14:
        return '8-14'
    elif SRCH_WINDOW >= 15 and SRCH_WINDOW <= 20:
        return '15-20'
    elif SRCH_WINDOW >= 21 and SRCH_WINDOW <= 30:
        return '21-30'
    elif SRCH_WINDOW >= 31 and SRCH_WINDOW <= 40:
        return '31-40'
    elif SRCH_WINDOW >= 41 and SRCH_WINDOW <= 60:
        return '41-60'
    elif SRCH_WINDOW >= 61 and SRCH_WINDOW <= 90:
        return '61-90'
    elif SRCH_WINDOW >= 91 and SRCH_WINDOW <= 120:
        return '91-120'
    elif SRCH_WINDOW >= 121 and SRCH_WINDOW <= 180:
        return '121-180'
    elif SRCH_WINDOW >= 181 and SRCH_WINDOW <= 240:
        return '181-240'
    else:
        return '>240'
    


In [21]:
#los bucket for booked los
def srchBookedLosBucket(booked_los):
    if booked_los is None:
        return '0'
    elif booked_los  in (0,1,2,3,4,5,6,7):
        return str(booked_los)
    elif booked_los >= 8 and booked_los <= 14:
        return '8-14'
    else:
        return '>14'

#bw bucket for booked bw
def srchBookedBwBucket(BOOKED_BKG_WINDOW):
    if BOOKED_BKG_WINDOW is None:
        return '0'
    elif BOOKED_BKG_WINDOW <= 0:
        return '<= 0'
    elif BOOKED_BKG_WINDOW == 1:
        return '1'
    elif BOOKED_BKG_WINDOW == 2:
        return '2'
    elif BOOKED_BKG_WINDOW >= 3 and BOOKED_BKG_WINDOW <= 4:
        return '3-4'
    elif BOOKED_BKG_WINDOW >= 5 and BOOKED_BKG_WINDOW <= 7:
        return '5-7'
    elif BOOKED_BKG_WINDOW >= 8 and BOOKED_BKG_WINDOW <= 14:
        return '8-14'
    elif BOOKED_BKG_WINDOW >= 15 and BOOKED_BKG_WINDOW <= 20:
        return '15-20'
    elif BOOKED_BKG_WINDOW >= 21 and BOOKED_BKG_WINDOW <= 30:
        return '21-30'
    elif BOOKED_BKG_WINDOW >= 31 and BOOKED_BKG_WINDOW <= 40:
        return '31-40'
    elif BOOKED_BKG_WINDOW >= 41 and BOOKED_BKG_WINDOW <= 60:
        return '41-60'
    elif BOOKED_BKG_WINDOW >= 61 and BOOKED_BKG_WINDOW <= 90:
        return '61-90'
    elif BOOKED_BKG_WINDOW >= 91 and BOOKED_BKG_WINDOW <= 120:
        return '91-120'
    elif BOOKED_BKG_WINDOW >= 121 and BOOKED_BKG_WINDOW <= 180:
        return '121-180'
    elif BOOKED_BKG_WINDOW >= 181 and BOOKED_BKG_WINDOW <= 240:
        return '181-240'
    else:
        return '>240'

In [22]:
#converting booking flag from string values to boolean
def booking_flag(booked_flag):
    if booked_flag == 'Y':
        return 1
    else:
        return 0

In [23]:
# Registering Functions as UDF's
udfsearchToCategory=udf(searchToCategory, StringType())
udfsearchbucketToCategory=udf(searchbucketToCategory, StringType())
udfsrchBookedLosBucket=udf(srchBookedLosBucket, StringType())
udfsrchsrchBookedBwBucket=udf(srchBookedBwBucket, StringType())
booking_flag_udf=udf(booking_flag, IntegerType())

In [24]:
from datetime import *
import time

date_list = ["2016-06-05",
"2016-06-06",
"2016-06-07",
"2016-06-08",
"2016-06-09",
"2016-06-10",
"2016-06-11"]


#schema for HCBA table
hcba_schema = StructType([
                            StructField('PARTNER_POS',StringType(), True),
                            StructField('DEVICE_TYPE',StringType(), True),
                            StructField('CLICKED_HOTEL_ID',IntegerType(), True),
                            StructField('SRCH_WINDOW',IntegerType(), True),
                            StructField('SRCH_LOS',IntegerType(), True),
                            StructField('TRAFFIC',IntegerType(), True),
                            StructField('COST',DoubleType(), True),
                            StructField('BOOKED_FLAG',StringType(), True),
                            StructField('BOOKED_HOTEL_ID',IntegerType(), True),
                            StructField('BOOKED_LOS',IntegerType(), True),
                            StructField('BOOKED_BKG_WINDOW',IntegerType(), True),
                            StructField('GROSS_PROFIT',DoubleType(), True),
                            StructField('PARTNER_ORG',StringType(), True),
                            StructField('ACTIVITY_DATE',StringType(), True)
                          ])

In [25]:
##CREATE EMPTY DATAFRAME 
hcba = sqlContext.createDataFrame(sc.emptyRDD(), hcba_schema)

##APPENDING THE DATA IN ABOVE CREATED hcba TABLE 
for dates in date_list:
    path_hcba = 's3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date='+dates+'/*'  
    
    try :
        temp_hcba = (sqlCtx.read.format("com.databricks.spark.csv").options(header = False, inferschema = True, delimiter = "\t",nullValue= '\\N')
                    .load(path_hcba)
                    .select('C1','C3','C7','C8','C9','C13','C14','C19','C20','C21','C22','C45','C65')
                    .filter("C65 = 'TRIPADVISOR' and C7 != '0' and C13 != '0'") 
                    .withColumn('ACTIVITY_DATE',lit(dates))
                    )
        hcba = hcba.unionAll(temp_hcba)
    except: 
        print (path_hcba)
        print('file does not exist')

In [26]:
## Changing Data Type of HCBA Table
cols_int = ['SRCH_LOS','SRCH_WINDOW','TRAFFIC','BOOKED_BKG_WINDOW','BOOKED_LOS']
cols_float = ['COST','GROSS_PROFIT']

hcba_changed_data_type = data_type_change_float(data_type_change_int(hcba,cols_int),cols_float)

hcba_final = (hcba_changed_data_type
                .withColumn("SRCH_LOS_BUCKET", udfsearchToCategory("SRCH_LOS"))
                .withColumn("SRCH_BW_BUCKET", udfsearchbucketToCategory("SRCH_WINDOW"))
                .withColumn("BOOKED_LOS_BUCKET", udfsrchBookedLosBucket("BOOKED_LOS"))
                .withColumn("BOOKED_BW_BUCKET", udfsrchsrchBookedBwBucket("BOOKED_BKG_WINDOW"))
                .withColumn("BOOKING", booking_flag_udf("BOOKED_FLAG"))
                .repartition(1)
             )


In [27]:
#columns required in final cleaned hcba table
cols_required = ['PARTNER_POS','DEVICE_TYPE','HOTEL_ID','PARTNER_ORG','ACTIVITY_DATE','TRAFFIC','BOOKING_WINDOW','LOS',
                 'COST','GROSS_PROFIT','LOS_BUCKET','BW_BUCKET','BOOKING']

In [28]:
#replacing the wrongly populated search parameters for the search queries that are booked with booking parameters
hcba_replac_srch_with_booking = (hcba_final.filter("BOOKED_FLAG = 'Y'")
                                 .filter("SRCH_WINDOW < -1 or SRCH_LOS <= 0")
                                 .withColumnRenamed('BOOKED_BKG_WINDOW', 'BOOKING_WINDOW')
                                 .withColumnRenamed('BOOKED_BW_BUCKET', 'BW_BUCKET')
                                 .withColumnRenamed('BOOKED_LOS_BUCKET', 'LOS_BUCKET')
                                 .withColumnRenamed('BOOKED_HOTEL_ID', 'HOTEL_ID')
                                 .withColumnRenamed('BOOKED_LOS', 'LOS')
                                 .select(cols_required)
                            )

In [29]:
#filtering out the outliers from the seacrh data
hcba_srch_data_cleaned = (hcba_final.filter("BOOKED_FLAG= 'N' and SRCH_WINDOW >= -1 and SRCH_LOS > 0 and GROSS_PROFIT = 0.0")
                             .withColumnRenamed('CLICKED_HOTEL_ID', 'HOTEL_ID')
                             .withColumnRenamed('SRCH_WINDOW', 'BOOKING_WINDOW')
                             .withColumnRenamed('SRCH_BW_BUCKET', 'BW_BUCKET')
                             .withColumnRenamed('SRCH_LOS', 'LOS')
                             .withColumnRenamed('SRCH_LOS_BUCKET', 'LOS_BUCKET')
                             .select(cols_required)
                            )


In [30]:
#filtering out the outilers for the booking data
hcba_booking_data_cleaned = (hcba_final.filter("BOOKED_FLAG= 'Y' and SRCH_WINDOW >= -1 and SRCH_LOS > 0")
                             .withColumnRenamed('CLICKED_HOTEL_ID', 'HOTEL_ID')
                             .withColumnRenamed('SRCH_WINDOW', 'BOOKING_WINDOW')
                             .withColumnRenamed('SRCH_BW_BUCKET', 'BW_BUCKET')
                             .withColumnRenamed('SRCH_LOS', 'LOS')
                             .withColumnRenamed('SRCH_LOS_BUCKET', 'LOS_BUCKET')
                             .select(cols_required)
                            )

In [31]:
#Union of all the above 3 tables 
hcba_cleaned_data =( hcba_replac_srch_with_booking.unionAll(hcba_srch_data_cleaned)
                    .unionAll(hcba_booking_data_cleaned)
                    .repartition(1)
                    .cache())

In [17]:
hcba_cleaned_data.select("PARTNER_POS").distinct().show()

+-----------+
|PARTNER_POS|
+-----------+
|         MX|
|         TH|
|         MY|
|         GR|
|         AE|
|         GT|
|         TR|
|         NG|
|         TW|
|         NL|
|         AR|
|         NO|
|        CO |
|         AT|
|         AU|
|         HK|
|         NZ|
|         UK|
|        CAR|
|         BE|
+-----------+
only showing top 20 rows



In [32]:
hcba_rm_gp0_book0 = hcba_cleaned_data.filter((hcba_cleaned_data.GROSS_PROFIT !=0.00) & (hcba_cleaned_data.BOOKING != 0) & \
                                 (hcba_cleaned_data.PARTNER_POS == 'CA') & (hcba_cleaned_data.DEVICE_TYPE=='DESKTOP') ).cache()

### Importing the data for all POS except CA

In [33]:
hcba_all_rm = hcba_cleaned_data.filter((hcba_cleaned_data.GROSS_PROFIT !=0.00) & (hcba_cleaned_data.BOOKING != 0) & \
                                 (hcba_cleaned_data.PARTNER_POS != 'CA') & (hcba_cleaned_data.DEVICE_TYPE=='DESKTOP') ).cache()

### agg at hotel_id , los  and bw level

In [50]:
test1 = hcba_rm_gp0_book0.groupBy("HOTEL_ID","BW_BUCKET","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})

test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [51]:
from pyspark.sql.functions import *
hcba_epb1 = test2.withColumn("EPB1",round("epb",2)).drop("epb").repartition(1).cache()

In [52]:
test1 = hcba_all_rm.groupBy("HOTEL_ID","BW_BUCKET","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})

test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [53]:
hcba_epb_all1 = test2.withColumn("EPB_all1",round("epb",2)).drop("epb").repartition(1).cache()

### agg at hotel_id and los level

In [55]:
test1 = hcba_rm_gp0_book0.groupBy("HOTEL_ID","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})

test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [56]:
hcba_epb2 = test2.withColumn("EPB2",round("epb",2)).drop("epb").repartition(1).cache()

In [99]:
test1 = hcba_all_rm.groupBy("HOTEL_ID","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})

test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [100]:
hcba_epb_all2 = test2.withColumn("EPB_all2",round("epb",2)).drop("epb").repartition(1).cache()

### agg at hotel_id level

In [59]:
test1 = hcba_rm_gp0_book0.groupBy("HOTEL_ID").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})

test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [60]:
hcba_epb3 = test2.withColumn("EPB3",round("epb",2)).drop("epb").repartition(1).cache()

In [61]:
test1 = hcba_all_rm.groupBy("HOTEL_ID").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})

test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [62]:
hcba_epb_all3 = test2.withColumn("EPB_all3",round("epb",2)).drop("epb").repartition(1).cache()

## joining with hotel_dim table to get market and other fields

In [63]:
path_hotel_dim = 's3n://ewe-core-meta-prod/CORE/HOTEL_DIM/*'
hotel_dim = sqlCtx.read.format("com.databricks.spark.csv").options(header = False, inferSchema = True, delimiter = "\t", nullValue= '\\N').			load(path_hotel_dim)
hotel_dim1 = hotel_dim.distinct().filter("C12 = 'BOOKABLE' or C12 ='NOT BOOKABLE' or C12 ='NOT APPLICABLE'").\
                select("C0","C7","C25","C26","C4","C13","C3")
hotel_dim4 = hotel_dim1.withColumnRenamed("C0","hotel_id").withColumnRenamed("C7" , "city").withColumnRenamed("C25","region").\
    withColumnRenamed("C26","market").withColumnRenamed("C4","brand").withColumnRenamed("C13","star_rating").\
    withColumnRenamed("C3","hotel_type")

#removing the records with hotel id 0
hotel_dim5 = hotel_dim4.filter((hotel_dim4['hotel_id'].isNotNull())).withColumnRenamed('hotel_id','HOTEL_ID').repartition(1).cache()


In [64]:
hcba_rm_gp0_book0_dim = hotel_dim5.join(broadcast(hcba_rm_gp0_book0),"HOTEL_ID","inner")

In [65]:
hcba_all_rm_dim = hcba_all_rm.join(hotel_dim5,"HOTEL_ID","inner").repartition(1).cache()

### agg at Market,Star_Rating,Hotel_Type,los bucket level

In [66]:
test1 = hcba_rm_gp0_book0_dim.groupBy("market","star_rating","hotel_type","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})

test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [67]:
hcba_epb4 = test2.withColumn("EPB4",round("epb",2)).drop("epb").repartition(1).cache()

In [68]:
test1 = hcba_all_rm_dim.groupBy("market","star_rating","hotel_type","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})

test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [69]:
hcba_epb_all4 = test2.withColumn("EPB_all4",round("epb",2)).drop("epb").repartition(1).cache()

### agg at Market,Star_Rating,Hotel_Type level

In [70]:
test1 = hcba_rm_gp0_book0_dim.groupBy("market","star_rating","hotel_type").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})

test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [71]:
hcba_epb5 = test2.withColumn("EPB5",round("epb",2)).drop("epb").repartition(1).cache()

In [72]:
test1 = hcba_all_rm_dim.groupBy("market","star_rating","hotel_type").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})

test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [73]:
hcba_epb_all5 = test2.withColumn("EPB_all5",round("epb",2)).drop("epb").repartition(1).cache()

### agg at Market level

In [74]:
test1 = hcba_rm_gp0_book0_dim.groupBy("market").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})

test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [75]:
hcba_epb51 = test2.withColumn("EPB51",round("epb",2)).drop("epb").repartition(1).cache()

In [76]:
test1 = hcba_all_rm_dim.groupBy("market").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})

test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [77]:
hcba_epb_all51 = test2.withColumn("EPB_all51",round("epb",2)).drop("epb").repartition(1).cache()

In [78]:
hcba_epb_all51.show(10)

+--------------------+---------+
|              market|EPB_all51|
+--------------------+---------+
|AMSTERDAM, NETHER...|    75.49|
|      MENORCA, SPAIN|   166.21|
| SINT MAARTEN, DUTCH|    30.84|
|BENDIGO, VIC, AUS...|     24.8|
|     HATTIESBURG, MS|    14.84|
|ROTORUA, NEW ZEALAND|    25.57|
|BRAZIL - BELO HOR...|     8.52|
|   TRAVERSE CITY, MI|     40.8|
|MOSCOW REGION, RU...|     7.18|
|              GAMBIA|   118.72|
+--------------------+---------+
only showing top 10 rows



### creating universe for optimization

In [79]:
from datetime import timedelta, date
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import *
import time

##fucntion for chaging the datatype
def data_type_change_int(df,cols):
    temp = df
    for col in cols:
        temp = temp.withColumn(col+'_1',df[col].cast(IntegerType())).drop(col).withColumnRenamed(col+'_1',col)
    return temp

def data_type_change_float(df,cols):
    temp = df
    for col in cols:
        temp = temp.withColumn(col+'_1',df[col].cast(DoubleType())).drop(col).withColumnRenamed(col+'_1',col)
    return temp

#star rating buckets
def srch_region_star_rating(star_rating):
    if star_rating == '2.0':
        return '2'
    elif star_rating == '2.5':
        return '2.5'
    elif star_rating == '3.0':
        return '3'
    elif star_rating == '3.5':
        return '3.5'
    elif star_rating == '4.0':
        return '4'
    elif star_rating == '1.0' or star_rating == '1.5':
        return '1-1.5'
    elif star_rating == '4.5' or star_rating == '5.0':
        return '4.5-5'
    else:
        return '0'

#udf for creating city buckets
def srch_region_city(city,model_name):
	if model_name == 'CANADA':
		if city in ('TORONTO' , 'NIAGARA FALLS' , 'MONTREAL' , 'VANCOUVER' , 'QUEBEC' , 'BANFF' , 'OTTAWA' , 'EDMONTON' , 'VICTORIA','CALGARY' , 'WHISTLER'):
			return city
		elif city in ('NIAGARA-ON-THE-LAKE','CANMORE','JASPER','KELOWNA','HALIFAX','MISSISSAUGA','LAKELOUISE','COLLINGWOOD','VERNON','PENTICTON','GATINEAU','DORVAL','MARKHAM'):
			return'G12'
		elif city in ('MONT-TREMBLANT','TOFINO','BLUEMOUNTAINS','OSOYOOS','UCLUELET','SAINT-SAUVEUR','WESTKELOWNA','LAMALBAIE','SAINTE-ADELE'):
			return'G13'
		elif city in ('WINNIPEG','LONDON','RICHMOND','BARRIE','REVELSTOKE','VAUGHAN','ORILLIA','BURNABY','SURREY'):
			return 'G14'
		else:
			return 'others'
	else:
		return 'NA'

#udf for hotel type
def srch_region_Hotel_type(Hotel_type,model_name):
	if model_name == 'CANADA':
		if Hotel_type in  ('HOTEL' , 'HOTEL RESORT' , 'MOTEL'):
			return Hotel_type
		elif Hotel_type in ('CONDOMINIUM RESORT' , 'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('LODGE' , 'BED & BREAKFAST' , 'HOSTEL/BACKPACKER ACCOMMODATION' , 'CABIN' , 'GUEST HOUSE'):
			return 'Budget_hotel'
		elif Hotel_type in ('APART-HOTEL', 'INN'):
			return 'Family_Premium'
		else:
			return 'Others'
	
	elif model_name == 'US':
		if Hotel_type in  ('HOTEL' , 'HOTEL RESORT' , 'MOTEL'):
			return Hotel_type
		elif Hotel_type in ('CONDOMINIUM RESORT' ,'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('LODGE' , 'BED & BREAKFAST' , 'HOSTEL/BACKPACKER ACCOMMODATION' , 'CABIN' , 'GUEST HOUSE'):
			return 'Budget_hotel'
		elif Hotel_type in ('APART-HOTEL' ,  'INN' , 'APARTMENT', 'VILLA', 'PRIVATE VACATION HOME'):
			return 'Family_Premium'
		else:
			return 'others'
		
	elif model_name == 'Mexico':
		if Hotel_type in  ('HOTEL' , 'HOTEL RESORT' , 'ALL-INCLUSIVE'):
			return Hotel_type
		elif Hotel_type in ('CONDOMINIUM RESORT' ,'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('LODGE' , 'BED & BREAKFAST' , 'HOSTEL/BACKPACKER ACCOMMODATION'):
			return 'Budget_hotel'
		elif Hotel_type in ('APART-HOTEL' ,'INN' , 'VILLA' , 'APARTMENT'):
			return 'Family_Premium'
		else:
			return 'others'
		
	elif model_name=="CARIBBEAN":
		if Hotel_type in ('HOTEL' , 'HOTEL RESORT' , 'ALL-INCLUSIVE') :
			return Hotel_type
		elif Hotel_type in ('CONDOMINIUM RESORT' ,'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('APART-HOTEL' ,'INN' , 'VILLA' , 'APARTMENT'):
			return 'Family_Premium'
		else:
			return 'others'
		
	elif model_name == 'Other Region':
		if Hotel_type in ('HOTEL' , 'HOTEL RESORT' , 'APART-HOTEL') :
			return Hotel_type
		elif Hotel_type in ('GUEST HOUSE' ,'BED & BREAKFAST', 'HOSTEL/BACKPACKER ACCOMMODATION', 'MOTEL' , 'HOSTAL (BUDGET HOTEL)'):
			return 'Budget_hotel'
		elif Hotel_type in ('APARTMENT','RESIDENCE','TOWNHOUSE','INN'):
			return 'Budget_hotel'
		else:
			return 'others'
	
	elif model_name=="North America":
		if Hotel_type in ('HOTEL' , 'MOTEL') :
			return Hotel_type
		elif Hotel_type in ('LODGE' ,  'BED & BREAKFAST' ,  'COTTAGE' ,  'GUEST HOUSE' ,  'CABIN'):
			return 'Budget_hotel'
		elif Hotel_type in ( 'APARTMENT' ,  'HOTEL RESORT' ,  'CONDO' ,  'INN' ,  'APART-HOTEL' ,  'CARAVAN PARK'):
			return 'Family_Premium'
		else:
			return 'others'
	else:
		return 'NOT APPLICABLE'

##udf for market bucket
def srch_region_market(market,model_name):
	if model_name == 'CANADA':
		if market in  ('TORONTO, ON, CAN' , 'NIAGARA FALLS, ON, CAN' , 'MONTREAL, QC, CAN' , 'BANFF AREA, AB, CAN' ,'VANCOUVER, BC, CAN' ,'QUEBEC CITY, QC, CAN' , 'EDMONTON, AB, CAN' , 'OTTAWA, ON, CAN' , 'OKANAGAN VALLEY, BC, CAN'):
			return market
		elif market in ('VICTORIA, BC, CAN' ,'CALGARY, AB, CAN' ,'JASPER AREA, AB, CAN'):
			return 'G10'
		elif market in ('WHISTLER, BC, CAN' ,'NIAGARA-ON-THE-LAKE, ON, CAN' ,'MONT TREMBLANT, QC, CAN' , 'UCLUELET, BC, CAN','TOFINO - UCLUELET, BC, CAN'):
			return 'G11'
		elif market in ('COLLINGWOOD, ON, CAN' , 'BC INTERIOR SKI, BC, CAN' ,'HALIFAX, NS, CAN','WINNIPEG, MB, CAN' ,'LONDON, ON, CAN' , 'KAMLOOPS, BC, CAN'):
			return 'G12'
		else:
			return 'others'
	
	elif model_name == 'US':
		return 'NA'
		
	elif model_name == 'Mexico':
		if market in  ('MEXICO - RIVIERA MAYA, PLAYA DEL CARMEN & TULUM' , 'MEXICO - CANCUN & ISLA MUJERES' ,'MEXICO - PUERTO VALLARTA, NUEVO VALLARTA & RIVIERA NAYARIT' , 'MEXICO - LOS CABOS' , 'CENTRAL AMERICA - COSTA RICA - GUANACASTE' ,'CENTRAL AMERICA - BELIZE' , 'CENTRAL AMERICA - COSTA RICA - PUNTARENAS' , 'CENTRAL AMERICA - COSTA RICA - OTHER' , 'MEXICO - COZUMEL''MEXICO - HUATULCO' , 'MEXICO - IXTAPA & ZIHUATANEJO' , 'CENTRAL AMERICA - HONDURAS' , 'CENTRAL AMERICA - PANAMA - OTHER') :
			return market
		elif market in ( 'CENTRAL AMERICA - COSTA RICA - SAN JOSE' , 'CENTRAL AMERICA - PANAMA - PANAMA CITY' ,'MEXICO - MEXICO CITY' ,'MEXICO - MEXICO CITY'						,'CENTRAL AMERICA - NICARAGUA','MEXICO - MAZATLAN','MEXICO - MANZANILLO','CENTRAL AMERICA - EL SALVADOR','MEXICO - ACAPULCO'):
			return 'hclt'
		elif market in ('CENTRAL AMERICA - NICARAGUA','MEXICO - MAZATLAN','MEXICO - MANZANILLO','CENTRAL AMERICA - EL SALVADOR',                        'MEXICO - ACAPULCO'):
			return 'lclt'
		else:
			return 'others'
		
	elif model_name=="CARIBBEAN":
		if market in ('PUNTA CANA, DOMINICAN REPUBLIC' , 'TURKS AND CAICOS' , 'MONTEGO BAY, JAMAICA' , 'ST. LUCIA' ,'OCHO RIOS, JAMAICA' ,                      'NEGRIL, JAMAICA' , 'BARBADOS'):
			return market
		elif market in ('ARUBA' , 'PUERTO PLATA, DOMINICAN REPUBLIC' , 'ANTIGUA AND BARBUDA'):
			return 'G8'
		elif market in ('NASSAU, BAHAMAS', 'DOMINICAN REPUBLIC ALL OTHER' , 'BERMUDA','BAHAMAS, OUT ISLANDS'):
			return 'G9'
		elif market in ('SAN JUAN, PUERTO RICO' ,'PARADISE ISLAND, BAHAMAS' ,'CAYMAN ISLANDS','SINT MAARTEN, DUTCH' ,'CURACAO','ST. MARTIN, FRENCH'):
			return 'G10'
		else:
			return 'others'
		
	elif model_name == 'Other Region':
		return 'NA'
	
	elif model_name=="North America":
		if  market in (  'GRAVENHURST - BRACEBRIDGE - HUNTSVILLE, ON, CAN','ONTARIO WEST, ON, CAN','ABBOTSFORD - CHILLIWACK, BC, CAN',                       'MAINE - COAST','ONTARIO EAST, ON, CAN','VANCOUVER ISLAND, BC, CAN','BURLINGTON, VT','CHARLOTTETOWN, PEI, CAN'): 
			return 'G1'
		elif market in ('ALBERTA EAST, AB, CAN','REGINA, SK, CAN','KINGSTON, ON, CAN','SASKATOON, SK, CAN','MONCTON, NB, CAN',                        'TRI-CITIES (KW-GUELPH-CAMBRIDGE), ON, CAN'): 
			return 'G2'
		elif market in ('WINDSOR, ON, CAN','RED DEER, AB, CAN','BRITISH COLUMBIA SOUTH, BC, CAN','GRAND FORKS, ND','SUDBURY, ON, CAN',                        'ONTARIO NORTH, ON, CAN','BANGOR, ME','HAMILTON - BRANTFORD, ON, CAN','BELLEVILLE - TRENTON - COBOURG, ON, CAN','FARGO, ND'): 
			return 'G3'
		elif market in ('LAKE PLACID, NY','PARKSVILLE, VANCOUVER ISLAND, BC, CAN','LAKE GEORGE, NY','OGUNQUIT, ME','EVERETT, WA',                        'COASTAL, BC, CAN','GROVE CITY, PA','BELLINGHAM, WA','NORTH CONWAY, NH','NEWFOUNDLAND OTHER, NL, CAN',                        'NOVA SCOTIA SOUTH, NS, CAN','PORTSMOUTH, NH'): 
			return 'G4'
		elif market in ('LETHBRIDGE, AB, CAN','FREDERICTON, NB, CAN','BRANDON, MB, CAN','ERIE, PA','NANAIMO, VANCOUVER ISLAND, BC, CAN',                        'SASKATCHEWAN SOUTH, SK, CAN','PRINCE GEORGE - QUESNEL, BC, CAN','MEDICINE HAT, AB, CAN','GRANDE PRAIRIE, AB, CAN',                        'ALBERTA NORTH, AB, CAN','DURHAM, ON, CAN','BRITISH COLUMBIA CENTRAL, BC, CAN','ONTARIO SOUTHWEST, ON, CAN',                        'SAULT STE. MARIE, ON, CAN','THUNDER BAY, ON, CAN','SAINT JOHN, NB, CAN','GREAT FALLS, MT','MINOT, ND (AREA)',                        'BRITISH COLUMBIA NORTH, BC, CAN','NORTH BAY, ON, CAN','PETERBOROUGH, ON, CAN','CAPE BRETON ISLAND, NS, CAN','ALBERTA WEST, AB, CAN',                        'HUDSON VALLEY - POUGHKEEPSIE, NY','FORT MCMURRAY, AB, CAN','NOVA SCOTIA NORTH, NS, CAN','WATERTOWN, NY','EDMUNSTON, NB, CAN',                        'OLYMPIC NATIONAL PARK AREA, WA','CEDAR CITY - BRYCE CANYON NATIONAL PARK, UT','SUMMERSIDE, PEI, CAN','SALEM, OR (AREA)','BUTTE - HELENA, MT'                        ,'BECKLEY, WV'): 
			return 'G5'
		elif market in ('SAGINAW, MI','WHITE MOUNTAINS, NH','FINGER LAKES NEW YORK','NEW YORK - WEST','NEW BRUNSWICK NORTH, NB, CAN',                        'SASKATCHEWAN NORTH, SK, CAN','MANITOBA SOUTH, MB, CAN','BAR HARBOR, ME','WASHINGTON NORTHWEST','VERMONT CENTRAL','ALBERTA SOUTH, AB, CAN',                        'WENATCHEE-LEAVENWORTH,WA','GANANOQUE, ON, CAN','PLATTSBURGH, (NY)','STRAITS OF MACKINAC, MI','SANDUSKY, OH','MANCHESTER, NH','ITHACA, NY',                        'MINNESOTA NORTHWEST','MISSOULA, MT (AREA)','YELLOWKNIFE, NT, CAN','PEI OTHER, PEI, CAN','BILLINGS, MT (AREA)','RAPID CITY - MOUNT RUSHMORE, SD',                        'ANDOVER, MA','YUMA, AZ','PORT HURON, MI','LANCASTER, PA','FORT LEE - PARAMUS, NJ','LEXINGTON, KY','MANITOBA NORTH, MB, CAN','ABERDEEN - OCEAN SHORES, WA',                        'CANADA TERRITORIES, YT, CAN','KNOXVILLE, TN','BURLINGTON - MOUNT VERNON, WA','FREDERICKSBURG, VA','ARIZONA SOUTHEAST',                        'WINDHAM COUNTY, VT','KENAI PENINSULA, AK','COLUMBIA, SC','CALIFORNIA DESERTS SOUTH'): 
			return 'G6'
		elif  market in ('ROME-UTICA, NY','SANDPOINT, ID (AREA)','MOAB - GREEN RIVER, UT','WHITEHORSE, YT, CAN','NEW YORK - NORTHEAST',                         'SARNIA, ON, CAN','AUGUSTA, ME','LAKE POWELL - GLEN CANYON NATIONAL PARK, AZ','NEW YORK - SOUTH','BISMARK, ND',                         'ALABAMA NORTH','ASHLAND - MEDFORD, OR','WASHINGTON NORTHEAST','THE BERKSHIRES, MA','EUGENE, OR (AREA)',                         'GRAND RAPIDS, MI','WASHINGTON SOUTHWEST','LAKE HAVASU CITY, AZ','SARATOGA, NY','WYTHEVILLE - VA',                         'ARCATA - EUREKA, CA','SOUTH CAROLINA EAST','CHATTANOOGA, TN','TOLEDO, OH','IDAHO FALLS, ID','WEST VIRGINIA SOUTH',                         'SPRINGFIELD, MA','OLYMPIA, WA','GETTYSBURG - PA'): 
			return 'G7'
		else:return 'Others'
	else:return 'NOT APPLICABLE'
		
def srch_model_name(region):
	if region == 'CANADA':
		return region
		
	elif region in ('FLORIDA (USA)','NEW YORK CITY','CALIFORNIA','GAMING (USA)','NORTHEAST (USA)','MOUNTAIN (USA)',                    'MIDWEST (USA)','CENTRAL (USA)','SF PACNW','ORLANDO','HAWAII'):
		return 'US'
		
	elif region in ("MEXICO - DOMESTIC REGION",'MEXICO - RESORTS REGION','CENTRAL AMERICA REGION'):
		return 'Mexico'
		
	elif region=="CARIBBEAN":
		return 'CARIBBEAN'
    
	elif region=="NORTH AMERICA REGIONAL TERRITORIES":
		return 'North America'
		
	else :
		return "Other Region"

def srch_region_bucket(region,model_name):
	if model_name == 'Other Region':
		if region in ('SPAIN & PORTUGAL' , 'ROME & ITALY RESORTS' , 'MID-ATLANTIC' , 'GREECE & TURKEY' , 'OCEANIA' , 'ITALY NORTH' ,                      'THAILAND' , 'MIDDLE EAST AND INDIAN OCEAN'):
			return region
		elif region in ('PARIS' , 'LONDON' , 'ENGLAND'):
			return 'G9'
		elif region in ('EUROPE REGIONAL TERRITORIES' , 'UK & IRELAND'):
			return 'G10'
		elif region in ('SOUTH AMERICA' , 'AFRICA AND EAST MED'):
			return 'G11'
		elif region in ('EASTERN EUROPE' , 'GERMANY, AUSTRIA & SWITZERLAND' , 'BENELUX' , 'FRANCE' , 'NORDIC'):
			return 'G12'
		elif region in ('JAPAN & MICRONESIA' , 'HONG KONG & MACAU' , 'SOUTHEAST ASIA EMERGING' , 'MAINLAND CHINA'):
			return 'G13'
		else:
			return 'others'
	else :
		return 'NA'


udfsrch_region_star_rating=udf(srch_region_star_rating, StringType())
udfsrch_region_city=udf(srch_region_city, StringType())
udfsrch_region_hotel_type=udf(srch_region_Hotel_type, StringType())
udfsrch_region_market=udf(srch_region_market, StringType())
udfsrch_model_name= udf(srch_model_name,StringType())
udfsrch_region_bucket = udf (srch_region_bucket,StringType())

hotel_dim_model_name = hotel_dim5.withColumn("MODEL_NAME", udfsrch_model_name("region"))
hotel_dim_bkts = (hotel_dim_model_name.withColumn("STAR_RATING_BUCKET", udfsrch_region_star_rating("star_rating")).
                  withColumn("HOTEL_TYPE_BUCKET", udfsrch_region_hotel_type("hotel_type","MODEL_NAME")).    
                  withColumn("MARKET_BUCKET",udfsrch_region_market("market","MODEL_NAME")).     
                  withColumn("REGION_BUCKET", udfsrch_region_bucket("region","MODEL_NAME"))      
                  .withColumn("CITY_BUCKET", udfsrch_region_city("city","MODEL_NAME"))
                  .repartition(1).
                 cache())



In [114]:
bw = ['<=0','1','2','3-4','5-7','8-14','15-20','21-30','31-40','41-60','61-90','91-120','121-180','181-240','>240']
los = ['0','1','2','3','4','5','6','7','8-14','>14']

cnt = 1

for i in range(len(los)):
    for j in range(len(bw)):
        if cnt == 1:
            sq_univ = hotel_dim_bkts.withColumn("LOS_BUCKET",lit(los[i])).withColumn("BW_BUCKET",lit(bw[j]))    
        else :
            temp = hotel_dim_bkts.withColumn("LOS_BUCKET",lit(los[i])).withColumn("BW_BUCKET",lit(bw[j]))
            sq_univ = sq_univ.unionAll(temp)
        cnt += 1
        
sq_univ_re = sq_univ.repartition(4).cache()

In [115]:
sq_univ1 = sq_univ_re.join((hcba_epb1),["HOTEL_ID","BW_BUCKET","LOS_BUCKET"],"left_outer")

In [116]:
sq_univ11 = sq_univ1.join((hcba_epb_all1),["HOTEL_ID","BW_BUCKET","LOS_BUCKET"],"left_outer")

In [117]:
sq_univ2 = sq_univ11.join(hcba_epb2,["HOTEL_ID","LOS_BUCKET"],"left_outer")

In [118]:
sq_univ21 = sq_univ2.join(hcba_epb_all2,["HOTEL_ID","LOS_BUCKET"],"left_outer")

In [119]:
sq_univ3 = sq_univ21.join(hcba_epb3,["HOTEL_ID"],"left_outer")

In [120]:
sq_univ31 = sq_univ3.join(hcba_epb_all3,["HOTEL_ID"],"left_outer")

In [121]:
sq_univ4 = sq_univ31.join(hcba_epb4,["market","star_rating","hotel_type","LOS_BUCKET"],"left_outer")

In [122]:
sq_univ41 = sq_univ4.join(hcba_epb_all4,["market","star_rating","hotel_type","LOS_BUCKET"],"left_outer")

In [123]:
sq_univ5 = sq_univ41.join(hcba_epb5,["market","star_rating","hotel_type"],"left_outer")

In [124]:
sq_univ51 = sq_univ5.join(hcba_epb_all5,["market","star_rating","hotel_type"],"left_outer")

In [125]:
sq_univ_coal = (sq_univ51.withColumn("EPB",coalesce('EPB1','EPB2','EPB3','EPB4','EPB5',
                                                      'EPB_all1','EPB_all2','EPB_all3','EPB_all4','EPB_all5')))
sq_univ_final= (sq_univ_coal.select([col for col in sq_univ_coal.columns if col not in 
                {'EPB1','EPB2','EPB3','EPB4','EPB5',
                'EPB_all1','EPB_all2','EPB_all3','EPB_all4','EPB_all5'} ])
               .cache())

In [126]:
sq_univ_final[sq_univ_final.EPB.isNull()].count()

21541200

In [127]:
sq_univ_final.count()

47554200

In [128]:
21541200/47554200

0.45298207098426635

In [129]:
sq_univ_final.write.parquet("s3n://ewe-meta-top/epb_universe_using_validationEPB/sq_univ_validEPB_set3")

In [124]:
sq_univ_coal[sq_univ_coal.EPB1.isNull() & sq_univ_coal.EPB2.isNull() & sq_univ_coal.EPB3.isNull()\
            & sq_univ_coal.EPB4.isNull() & sq_univ_coal.EPB5.isNull() & sq_univ_coal.EPB51.isNull()\
            & sq_univ_coal.EPB6.isNull() & sq_univ_coal.EPB7.isNull() & sq_univ_coal.EPB8.isNull()\
            & sq_univ_coal.EPB9.isNull() & sq_univ_coal.EPB10.isNull() & sq_univ_coal.EPB11.isNull()
            & sq_univ_coal.EPB_all1.isNull() & sq_univ_coal.EPB_all2.isNull() & sq_univ_coal.EPB_all3.isNull()
            & sq_univ_coal.EPB_all4.isNull()  & sq_univ_coal.EPB_all5.isNull() & sq_univ_coal.EPB_all51.isNull()
            & sq_univ_coal.EPB_all6.isNull() & sq_univ_coal.EPB_all7.isNull() & sq_univ_coal.EPB_all8.isNull()
            & sq_univ_coal.EPB_all9.isNull() & sq_univ_coal.EPB_all10.isNull() & sq_univ_coal.EPB_all11.isNull()].count()

5139817

In [53]:
#new one 
16002508/38691058

0.41359706421054704

In [96]:
#new one 
5139817/38691058

0.13284250329882424

In [50]:
old one
17252315/38685724

0.4459607632003992

### saving all epb tables in s3

In [ ]:
#hcba_epb1.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb1.csv')

In [ ]:
'''
hcba_epb2.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb2.csv')
hcba_epb3.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb3.csv')
hcba_epb4.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb4.csv')
hcba_epb5.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb5.csv')
hcba_epb6.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb6.csv')
'''

In [36]:
'''
hcba_epb7.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb7.csv')
hcba_epb8.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb8.csv')
hcba_epb9.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb9.csv')
hcba_epb10.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb10.csv')
hcba_epb11.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb11.csv')
'''

# EDA

In [9]:
hcba1.select("ACTIVITY_DATE").distinct().show(400)

+-------------+
|ACTIVITY_DATE|
+-------------+
|   2015-12-27|
|   2015-12-28|
|   2015-11-20|
|   2015-12-29|
|   2015-11-21|
|   2015-11-22|
|   2015-11-23|
|   2015-11-24|
|   2015-11-25|
|   2015-11-26|
|   2015-09-20|
|   2015-11-27|
|   2015-09-21|
|   2015-11-28|
|   2015-09-22|
|   2015-10-20|
|   2015-11-29|
|   2015-09-23|
|   2015-10-21|
|   2015-09-24|
|   2015-10-22|
|   2015-09-25|
|   2015-10-23|
|   2015-09-26|
|   2015-10-24|
|   2015-09-27|
|   2015-10-25|
|   2015-09-28|
|   2015-10-26|
|   2015-08-20|
|   2015-09-29|
|   2015-10-27|
|   2015-08-21|
|   2015-10-28|
|   2015-08-22|
|   2015-10-29|
|   2015-08-23|
|   2015-08-24|
|   2015-08-25|
|   2015-12-30|
|   2015-08-26|
|   2015-12-31|
|   2015-08-27|
|   2015-08-28|
|   2015-08-29|
|   2015-11-30|
|   2015-09-30|
|   2015-10-30|
|   2015-10-31|
|   2015-08-30|
|   2015-08-31|
|   2016-06-01|
|   2016-06-02|
|   2016-06-03|
|   2016-06-04|
|   2016-06-05|
|   2016-06-06|
|   2016-06-07|
|   2016-06-08|
|   2016

In [11]:
hcba1.agg({"BOOKING":"sum","TRAFFIC":"sum"}).show()

+------------+------------+
|sum(TRAFFIC)|sum(BOOKING)|
+------------+------------+
|   6182082.0|    131061.0|
+------------+------------+



In [12]:
hcba1.describe("GROSS_PROFIT").show()

+-------+------------------+
|summary|      GROSS_PROFIT|
+-------+------------------+
|  count|           4017467|
|   mean| 1.368504559962782|
| stddev|14.952093137318132|
|    min|           -0.0126|
|    max|          998.2305|
+-------+------------------+



In [13]:
hcba1[hcba1.GROSS_PROFIT==0.00].count()

3889597

In [14]:
hcba1.count()

4017467

In [15]:
hcba1[hcba1.BOOKING==0].count()

3886406

In [3]:
hcba_epb1_temp = hcba1.groupBy("HOTEL_ID","LOS_BUCKET","BW_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})

In [4]:
from pyspark.sql.functions import *

hcba_epb1_temp1 = hcba_epb1_temp.withColumn("EPB_1",(hcba_epb1_temp["sum(GROSS_PROFIT)"]/hcba_epb1_temp["sum(BOOKING)"])).drop("sum(GROSS_PROFIT)").\
                drop("sum(BOOKING)")
    
hcba_epb1=hcba_epb1_temp1.withColumn("EPB",round(hcba_epb1_temp1.EPB_1,2)).drop("EPB_1")

In [31]:
hcba_epb1[hcba_epb1.EPB==0.00].count()

1985

In [32]:
hcba_epb1[hcba_epb1.EPB.isNull()].count()

1334339

In [34]:
hcba_epb1[hcba_epb1.EPB.isNull()].show(10)

+--------+----------+---------+----+
|HOTEL_ID|LOS_BUCKET|BW_BUCKET| EPB|
+--------+----------+---------+----+
|  104193|         2|   91-120|null|
|    1755|         2|   91-120|null|
|   20611|         4|     8-14|null|
|   24013|         4|     8-14|null|
| 3847536|         2|   91-120|null|
| 4199599|      8-14|  121-180|null|
| 6084581|         7|    31-40|null|
|  788618|         7|    31-40|null|
|  796475|         7|    31-40|null|
|   18561|         2|  181-240|null|
+--------+----------+---------+----+
only showing top 10 rows



In [37]:
hcba[hcba.GROSS_PROFIT==0.00].count()

3889597

In [38]:
hcba[hcba.BOOKING==0].count()

3886406

In [24]:
hcba_epb1.count()

1436482

In [27]:
hcba_epb1_temp1 = hcba1.filter("ACTIVITY_DATE<'2016-06-07'").groupBy("HOTEL_ID","LOS_BUCKET","BW_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})

from pyspark.sql.functions import *

hcba_epb1_temp2 = hcba_epb1_temp1.withColumn("EPB_1",(hcba_epb1_temp1["sum(GROSS_PROFIT)"]/hcba_epb1_temp1["sum(BOOKING)"])).drop("sum(GROSS_PROFIT)").\
                drop("sum(BOOKING)")
test_epb=hcba_epb1_temp2.withColumn("EPB",round(hcba_epb1_temp2.EPB_1,2)).drop("EPB_1")

In [28]:
valid = hcba1.filter("ACTIVITY_DATE>='2016-06-07'")

In [ ]:
test_epb

In [29]:
test_epb[~test_epb.EPB.isNull()].join(valid,["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],"inner").count()

12273

In [9]:
valid.count()

84450

In [11]:
hcba_epb1_temp1 = hcba1.filter("ACTIVITY_DATE<'2016-06-07'").groupBy("HOTEL_ID","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})

from pyspark.sql.functions import *

hcba_epb1_temp2 = hcba_epb1_temp1.withColumn("EPB_1",(hcba_epb1_temp1["sum(GROSS_PROFIT)"]/hcba_epb1_temp1["sum(BOOKING)"])).drop("sum(GROSS_PROFIT)").\
                drop("sum(BOOKING)")
test_epb2=hcba_epb1_temp2.withColumn("EPB2",round(hcba_epb1_temp2.EPB_1,2)).drop("EPB_1")

In [33]:
test_epb2[~test_epb2.EPB2.isNull()].join(valid,["HOTEL_ID","LOS_BUCKET"],"inner").count()

36562

In [21]:
v1 = valid.join(test_epb[~test_epb.EPB.isNull()],["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],"left_outer")
v2 = v1.join(test_epb2[~test_epb2.EPB2.isNull()],["HOTEL_ID","LOS_BUCKET"],"left_outer")

In [22]:
v3 = v2.withColumn("EPB_final",coalesce("EPB","EPB2"))

In [23]:
v3[~v3.EPB_final.isNull()].count()

36562

In [19]:
v3.count()

84450

In [24]:
v1[~v1.EPB.isNull()].count()

12273

In [25]:
v2[~v2.EPB2.isNull()].count()

36562

In [26]:
v2[v2.EPB2.isNull()][v2.EPB.isNull()].count()

47888

In [ ]:
hcba_remove_gp0_book0 = hcba1.filter("GROSS_PROFIT !=0.00 and BOOKING != 0 and ACTIVITY_DATE <= 2016-06-06")